<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h3>Formating the data</h3>

Import Libraries and get url content

In [229]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

Extract table and move to a DataFrame

In [274]:
table = html_soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')
table_columns = table.find_all('th')
type(table_columns)
#Get columns name
c = []
for th in table_columns:
    text = th.text.replace('\n', '')
    c.append(text)
    
#Get rows
r = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if(row == []):
        continue
    row = [element.strip('\n') for element in row]
    r.append(row)

df = pd.DataFrame(r, columns=c)
df.shape

(289, 3)

Remove not assigned boroughs

In [275]:
df = df[df['Borough'] != 'Not assigned']

Substitute not assigned neighbourhoods for borough

In [276]:
for i,row in df.iterrows():
    if (row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

Combine Neighborhood in Postcodes

In [278]:
df_aux = df[['Postcode', 'Borough', 'Neighbourhood']]
df_aux = df_aux.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index()

df = pd.merge(df, df_aux, on=['Postcode'], how='inner')
df = df[['Postcode','Borough','Neighbourhood_y']]
df=df.rename(columns = {'Neighbourhood_y':'Neighbourhood'})
df[:5]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [279]:
df.shape

(212, 3)